In [29]:
from openqaoa.problems import QUBO
from openqaoa import QAOA

From [QUBO's tutorial paper](https://arxiv.org/pdf/1811.11538.pdf)
\begin{align}
y = -5x_0 -3x_1 -8x_2 -6x_3 +4x_0x_1 +8x_0x_2 +2x_1x_2 +10x_2x_3
\end{align}

\begin{align}
        y =
        (x_0 \; x_1 \; x_2 \; x_3)
        \begin{pmatrix}
            -5 & 2 & 4 & 0\\ 
            2 & -3 & 1 & 0\\
            4 & 1 & -8 & 5\\
            0 & 0 & 5 & -6
        \end{pmatrix}
        \begin{pmatrix}
            x_0 \\ x_1 \\ x_2 \\ x_3
        \end{pmatrix}
\end{align}

In [23]:
terms = [[0], [1], [2], [3], [0,1], [0,2], [1,2], [2,3]]
weights = [-5, -3, -8, -6, 4, 8, 2, 10]

qubo1 = QUBO(n=4, terms=terms, weights=weights)
qubo1.asdict()

{'terms': [[0], [1], [2], [3], [0, 1], [0, 2], [1, 2], [2, 3]],
 'weights': [-5.0, -3.0, -8.0, -6.0, 4.0, 8.0, 2.0, 10.0],
 'constant': 0,
 'n': 4,
 'problem_instance': {'problem_type': 'generic_qubo'},
 'metadata': {}}

The solution according to paper: $y$ = -11, $x_0$ = $x_3$ = 1, $x_1$ = $x_2$ = 0, so bitstring is 1001.

Solving the problem with QAOA with default configuration:

In [24]:
s1 = QAOA()
s1.compile(qubo1)
s1.optimize()
result = s1.result

In [22]:
result.optimized

{'angles': [1.243729362954, 0.340245636307],
 'cost': -1.720743265023,
 'measurement_outcomes': array([-0.12864806+0.10612882j, -0.09740045+0.21169534j,
        -0.03200846+0.03879354j, -0.03902651+0.14660317j,
        -0.12930469-0.11503485j,  0.00699163-0.07406441j,
         0.160736  +0.29802834j, -0.0941285 +0.14185505j,
         0.21362316+0.10500403j,  0.44907972+0.06726187j,
         0.07424044+0.23259368j,  0.19794531+0.05813571j,
         0.16813648+0.19127032j,  0.05549009+0.00557399j,
         0.02316411-0.43732588j,  0.24970117-0.16148j   ]),
 'job_id': 'e6b23aa7-1e73-4c10-8bde-8c65c1a1c46c',
 'eval_number': 100}

In [25]:
result.most_probable_states

{'solutions_bitstrings': ['1001'], 'bitstring_energy': -20.0}

So it does't match -11 but solutions_bitstrings is exactly 1001. So let's try another tool such as qubovert.

In [7]:
import qubovert as qv

In [8]:
Q = [[-5, 2, 4, 0],
     [2, -3, 1, 0],
     [4, 1, -8, 5], 
     [0, 0, 5, -6]]

In [26]:
q2 = qv.utils.matrix_to_qubo(Q)
s2 = qv.utils.solve_qubo_bruteforce(q2)

In [27]:
s2

(-11, {0: 1, 1: 0, 2: 0, 3: 1})

Here we got the exact solution as in paper.

Why do we have different results for y? -20 (OpenQAOA) vs -11 (qubovert)